In [1]:
import re
import os
import subprocess
source = "IITH"
destination = ["google.txt","alibaba.txt","apple.txt","bitcoin.txt","goldmansachs.txt","ntt.txt","spotify.txt","vodacom.txt","wikipedia.txt","oracle.txt"]
# destination = ["nvidia.txt"]
def getdata(source,destination):
    file = open(source+"/"+destination,"r")
    data = {
        "AS NUMBER" : [],
        "IP ADDRESS": [],
        "IP RANGE" : [],
        "LOCATION" : [],
        "ORGANISATION": []
    }
    for line in file.readlines():

    
        s = re.findall('\[AS[0-9]+',line)
        
        if (len(s)>0 and s[0][1] != "*"):
            
            ips = re.findall('[0-9]+\.[0-9]+\.[0-9]+\.[0-9]+',line)

            statement = "./IPGeoLocation/ipgeolocation.py -t {} ".format(ips[0])
            proc = subprocess.Popen([statement],stdout=subprocess.PIPE,shell=True)
            (out,err) = proc.communicate()
            out = out.decode()
            city = re.findall('City:[^\n]*\\n',out)
            city = re.findall('[A-Z][a-zA-Z ]*',city[0])
            country= re.findall('Country:[^\n]*\\n',out)
            country = re.findall('[A-Z][a-zA-Z ]*',country[0])
            org= re.findall('Organization:[^\n]*\\n',out)
            org = re.findall('[A-Z][a-zA-Z ]*',org[0])
            if (len(org) >=2):
                org = org[1]
            else:
                org = ""
            if (len(city) >=2):
                city = city[1]
            else:
                city = ""
            if (len(country) >=2):
                country = country[1]
            else:
                country = ""
            
            loc = city+'/'+country
            statement = "whois -a {}".format(ips[0])
            proc2 = subprocess.Popen([statement],stdout=subprocess.PIPE,shell=True)
            (out,err) = proc2.communicate()
            out = out.decode()
            iprange = re.findall('inetnum:[ \t]*[-0-9\. ]*|NetRange:[ \t]*[-0-9\. ]*',out)
            if (len(iprange) > 0):
                iprange = iprange[0].split(":")[1].lstrip()
            else:
                iprange = ""
                
            
            data["IP RANGE"].append(iprange)
            data["AS NUMBER"].append(s[0][1:])
            data["IP ADDRESS"].append(ips[0])
            data["ORGANISATION"].append(org)
            data["LOCATION"].append(loc)
            
    file.close()
    return data


In [2]:
# getdata(source,destination[0])
destination_dict = {}
for d in destination:
    print(d)
    destination_dict[d] = getdata(source,d)
destination_dict

google.txt
alibaba.txt
apple.txt
bitcoin.txt
goldmansachs.txt
ntt.txt
spotify.txt
vodacom.txt
wikipedia.txt
oracle.txt


{'google.txt': {'AS NUMBER': ['AS15169', 'AS15169', 'AS15169'],
  'IP ADDRESS': ['72.14.195.128', '142.250.228.81', '142.250.67.46'],
  'IP RANGE': ['72.14.192.0 - 72.14.255.255',
   '142.250.0.0 - 142.251.255.255',
   '142.250.0.0 - 142.251.255.255'],
  'LOCATION': ['Mountain View/United States',
   'Mountain View/United States',
   'Laurelton/United States'],
  'ORGANISATION': ['Google LLC', 'Google LLC', 'Google LLC']},
 'alibaba.txt': {'AS NUMBER': ['AS59193', 'AS55836', 'AS3356', 'AS45102'],
  'IP ADDRESS': ['103.232.241.2',
   '115.247.100.29',
   '4.69.220.22',
   '47.246.137.166'],
  'IP RANGE': ['103.232.241.0 - 103.232.241.255',
   '115.240.0.0 - 115.247.255.255',
   '4.0.0.0 - 4.127.255.255',
   '47.235.0.0 - 47.246.255.255'],
  'LOCATION': ['Hyderabad/India',
   'Vijayawada/India',
   'New York/United States',
   'Ashburn/United States'],
  'ORGANISATION': ['',
   'Reliance Communication Ltd',
   'Level ',
   'Alibaba Cloud LLC']},
 'apple.txt': {'AS NUMBER': ['AS59193',
  

In [3]:
destination_dict

{'google.txt': {'AS NUMBER': ['AS15169', 'AS15169', 'AS15169'],
  'IP ADDRESS': ['72.14.195.128', '142.250.228.81', '142.250.67.46'],
  'IP RANGE': ['72.14.192.0 - 72.14.255.255',
   '142.250.0.0 - 142.251.255.255',
   '142.250.0.0 - 142.251.255.255'],
  'LOCATION': ['Mountain View/United States',
   'Mountain View/United States',
   'Laurelton/United States'],
  'ORGANISATION': ['Google LLC', 'Google LLC', 'Google LLC']},
 'alibaba.txt': {'AS NUMBER': ['AS59193', 'AS55836', 'AS3356', 'AS45102'],
  'IP ADDRESS': ['103.232.241.2',
   '115.247.100.29',
   '4.69.220.22',
   '47.246.137.166'],
  'IP RANGE': ['103.232.241.0 - 103.232.241.255',
   '115.240.0.0 - 115.247.255.255',
   '4.0.0.0 - 4.127.255.255',
   '47.235.0.0 - 47.246.255.255'],
  'LOCATION': ['Hyderabad/India',
   'Vijayawada/India',
   'New York/United States',
   'Ashburn/United States'],
  'ORGANISATION': ['',
   'Reliance Communication Ltd',
   'Level ',
   'Alibaba Cloud LLC']},
 'apple.txt': {'AS NUMBER': ['AS59193',
  

In [129]:
import pandas as pd

for d in destination:
    df = pd.DataFrame(destination_dict[d])
    df.to_excel(source +"/"+"structured" + "/" + d[:-3]+"xlsx")